### 1. 라이브러리 설치

In [1]:
import pandas as pd
import re

### 2. 파일 불러오기

In [2]:
patent = pd.read_csv('../1. 데이터 수집/patent_crawling/patent_list_final.csv')

# DART API 누락데이터 6건 삭제 ('아리온', 'JTC', '패션플랫폼', '강원에너지', '무림SP', '코센')
patent = patent[(patent.corp_name !='아리온') & (patent.corp_name !='JTC') & (patent.corp_name !='패션플랫폼') &\
    (patent.corp_name !='강원에너지') & (patent.corp_name !='무림SP') & (patent.corp_name !='코센')]

# patent.to_excel('C:/Users/Chae/Desktop/SeSAC/Project/5. ESG_prj/1. 데이터 수집/patent_crawling/patent_list_final.xlsx', index=False)

### 3. 크롤링 에러 확인
- error 사유 : 검색결과 없음   >>> patent '0건' 으로 처리
- 117+204 = 321건

In [3]:
# error_list1 = pd.read_csv('../1. 데이터 수집/patent_crawling/error_list.csv')
# error_list2 = pd.read_csv('../1. 데이터 수집/patent_crawling/error_list2.csv')
# error_merged = pd.concat([error_list1, error_list2], axis=0)
# error_merged

# error_merged.to_excel('C:/Users/Chae/Desktop/SeSAC/Project/5. ESG_prj/1. 데이터 수집/patent_crawling/patent_error.xlsx', index=False)

### 4. 컬럼별 전처리
- corp_num : Key값
- corp_name
- count : 조회건수 확인용이므로 제거
- company_name : corp_name과 중복이므로 제거
- title : 행정상태(맨앞2자리)가 '공개', '등록'만 남기기
- text : null값 row 제거
- date : 날짜형식으로 변경 (0000-00-00)

date 컬럼

In [4]:
# 연도만 남기기
patent['date'] = patent['date'].str.split('(').str[1]
patent['date'] = patent['date'].str[0:4]
patent['date']

# patent.replace({'date': {'\)':''}}, inplace=True, regex=True)
# patent.replace({'date': {'\.':'-'}}, inplace=True, regex=True)

0        2018
1        2018
2        2013
3        2011
4        2012
         ... 
88555    1995
88556    1995
88557    1995
88558    1993
88559    1993
Name: date, Length: 88556, dtype: object

text 컬럼

In [5]:
# null인 경우, 실제 값도 '내용 없음' 이며 특허등록 상태가 아님 -> 498건 삭제
patent = patent.dropna(subset=['text'])
patent

,corp_num,corp_name,count,company_name,title,text,date
0,6408701749,블리츠웨이,2,주식회사 블리츠웨이,등록 [1] 흉부 움직임이 가능한 피규어(Figures that can move c...,"본 발명은 인간, 로봇 및 동물의 몸통을 나타내고, 사용자의 조작에 의해 흉부를 움...",2018
1,6408701749,블리츠웨이,2,주식회사 블리츠웨이,등록 [2] 길이 조절이 가능한 피규어(Length adjustable figure),"본 발명은 인간, 로봇 및 동물의 몸통을 나타내는 몸통부재; 인간, 로봇 및 동물의...",2018
2,1348163122,석경에이티,75,한양대학교 산학협력단,등록 [1] 기능기를 갖는 실리카 입자의 제조방법 및 이에 따라 제조된 실리카 입자...,본 발명은 기능기를 갖는 실리카 입자의 제조방법 및 이에 따라 제조된 실리카 입자에...,2013
3,1348163122,석경에이티,75,(주)석경에이티,등록 [2] 단분산 고순도 실리카 미세입자의 제조방법(PREPARATION METH...,본 발명은 실리카 전구체를 박막증발기를 이용하여 분별증류하는 단계; 및 상기 분별증...,2011
4,1348163122,석경에이티,75,(주)석경에이티,등록 [3] 연마제가 코팅된 자성입자의 제조방법 및 자기 연마 필러(Preparat...,본 발명은 (a) 모입자인 판상의 자성 입자를 함유하는 수용액에 자입자인 연마제를 ...,2012
...,...,...,...,...,...,...,...
88555,1278108271,카스,100,주식회사 카스,소멸 [96] 염화이온센서의전극에구비되는이온선택성막을형성하기위한조성물및이것을사용하여...,본 발명은 염화이온 센서의 전극에 구비되는 염화이온 선택성 막을 형성하기 위한 조성...,1995
88556,1278108271,카스,100,주식회사 카스,소멸 [97] 반도체형용존가스센서(SEMICONDUCTOR TYPE DISSOLVE...,본 발명은 수용액 속에 용해되어 있는 가스의 농도 또는 분압을 측정하는 반도체형 용...,1995
88557,1278108271,카스,100,주식회사 카스이앤씨,소멸 [98] 이온감지막을형성하기위한조성물및이온감지막을구비하는고체형이온센서(COMP...,"본 발명은 고체형 이온 센서에 관한 것으로서, 특히 종래의 재래식 전극형 이온 센서...",1995
88558,1278108271,카스,100,주식회사 카스,소멸 [99] 기준전계효과트랜지스터와그제조방법(REFERENCE FIELD EFFE...,요약 없슴.,1993


In [6]:
# 텍스트 전처리
def preprocessing(str):
    # 한글, 공백 빼고 다 삭제
    str = re.sub('[^가-힣\s]','',str)
    return str

patent['text'] = patent['text'].apply(preprocessing)

title 컬럼

In [7]:
# 맨앞2자리는 행정상태를 나타냄
# '공개', '등록' -> 저장
# '취하', '소멸', '포기', '무효', '거절' -> 삭제

# state 컬럼 생성
patent['state'] = patent['title'].str[0:2]
print(patent['state'].value_counts(), '\n')

drops = ['취하', '소멸', '포기', '무효', '거절']
idx = []
for i in drops:
    idx = patent[patent['state']==i].index
    # 해당조건 행 삭제
    patent.drop(idx, inplace=True)

# state 컬럼 삭제
patent = patent.drop(['state'], axis=1)

등록    44602
소멸    22146
거절    11169
공개     4927
취하     4571
포기      639
무효        4
Name: state, dtype: int64 



In [8]:
# 텍스트 전처리
patent['title'] = patent['title'].apply(preprocessing)
# 행정상태 단어 제거
patent['title'] = patent['title'].str[2:]
patent['title']

0                                      흉부 움직임이 가능한 피규어    
1                                         길이 조절이 가능한 피규어  
2          기능기를 갖는 실리카 입자의 제조방법 및 이에 따라 제조된 실리카 입자     ...
3                           단분산 고순도 실리카 미세입자의 제조방법        
4             연마제가 코팅된 자성입자의 제조방법 및 자기 연마 필러              
                               ...                        
88501                    전문가용 디지털 체중 관리 시스템 및 그 방법        
88504                      체중기반 건강관리방법 및 그 기록매체           
88506                                      공중용 디지털 체중계    
88511                                        차량 중량 측정기    
88521                          자동강수샘플러 시스템 및 그 작동방법       
Name: title, Length: 49529, dtype: object

컬럼 정리

In [9]:
patent

,corp_num,corp_name,count,company_name,title,text,date
0,6408701749,블리츠웨이,2,주식회사 블리츠웨이,흉부 움직임이 가능한 피규어,본 발명은 인간 로봇 및 동물의 몸통을 나타내고 사용자의 조작에 의해 흉부를 움직일...,2018
1,6408701749,블리츠웨이,2,주식회사 블리츠웨이,길이 조절이 가능한 피규어,본 발명은 인간 로봇 및 동물의 몸통을 나타내는 몸통부재 인간 로봇 및 동물의 팔을...,2018
2,1348163122,석경에이티,75,한양대학교 산학협력단,기능기를 갖는 실리카 입자의 제조방법 및 이에 따라 제조된 실리카 입자 ...,본 발명은 기능기를 갖는 실리카 입자의 제조방법 및 이에 따라 제조된 실리카 입자에...,2013
3,1348163122,석경에이티,75,(주)석경에이티,단분산 고순도 실리카 미세입자의 제조방법,본 발명은 실리카 전구체를 박막증발기를 이용하여 분별증류하는 단계 및 상기 분별증류...,2011
4,1348163122,석경에이티,75,(주)석경에이티,연마제가 코팅된 자성입자의 제조방법 및 자기 연마 필러,본 발명은 모입자인 판상의 자성 입자를 함유하는 수용액에 자입자인 연마제를 넣은 ...,2012
...,...,...,...,...,...,...,...
88501,1278108271,카스,100,주식회사 카스,전문가용 디지털 체중 관리 시스템 및 그 방법,본 발명은 전문적으로 건강을 관리해주는 헬스클럽 비만 클리닉 등에서 사용할 수 있는...,2003
88504,1278108271,카스,100,주식회사 카스,체중기반 건강관리방법 및 그 기록매체,본 발명은 개인별 특성을 고려하여 식사량이나 운동량을 적절하게 조절하여 체중을 관리...,2003
88506,1278108271,카스,100,주식회사 카스,공중용 디지털 체중계,본 발명은 공중용 디지털 체중계에 관한 것으로 이러한 본 발명은 하중감지수단 하중감...,2003
88511,1278108271,카스,100,주식회사 카스,차량 중량 측정기,본 발명은 차량 중량 측정기에 관한 것으로서 더욱 상세하게는 차량의 진입이 용이하여...,2019


In [10]:
# 불필요한 컬럼 삭제
patent = patent.drop(['corp_num', 'count', 'company_name'], axis=1)
# 컬럼명 변경
patent = patent.rename(columns={'corp_name':'cmpID', 'title':'Title', 'text':'Content', 'date':'Year'})

In [11]:
patent

,cmpID,Title,Content,Year
0,블리츠웨이,흉부 움직임이 가능한 피규어,본 발명은 인간 로봇 및 동물의 몸통을 나타내고 사용자의 조작에 의해 흉부를 움직일...,2018
1,블리츠웨이,길이 조절이 가능한 피규어,본 발명은 인간 로봇 및 동물의 몸통을 나타내는 몸통부재 인간 로봇 및 동물의 팔을...,2018
2,석경에이티,기능기를 갖는 실리카 입자의 제조방법 및 이에 따라 제조된 실리카 입자 ...,본 발명은 기능기를 갖는 실리카 입자의 제조방법 및 이에 따라 제조된 실리카 입자에...,2013
3,석경에이티,단분산 고순도 실리카 미세입자의 제조방법,본 발명은 실리카 전구체를 박막증발기를 이용하여 분별증류하는 단계 및 상기 분별증류...,2011
4,석경에이티,연마제가 코팅된 자성입자의 제조방법 및 자기 연마 필러,본 발명은 모입자인 판상의 자성 입자를 함유하는 수용액에 자입자인 연마제를 넣은 ...,2012
...,...,...,...,...
88501,카스,전문가용 디지털 체중 관리 시스템 및 그 방법,본 발명은 전문적으로 건강을 관리해주는 헬스클럽 비만 클리닉 등에서 사용할 수 있는...,2003
88504,카스,체중기반 건강관리방법 및 그 기록매체,본 발명은 개인별 특성을 고려하여 식사량이나 운동량을 적절하게 조절하여 체중을 관리...,2003
88506,카스,공중용 디지털 체중계,본 발명은 공중용 디지털 체중계에 관한 것으로 이러한 본 발명은 하중감지수단 하중감...,2003
88511,카스,차량 중량 측정기,본 발명은 차량 중량 측정기에 관한 것으로서 더욱 상세하게는 차량의 진입이 용이하여...,2019


### 5. 2020년, 2021년 나누기

In [18]:
# 2020년 기준
patent_2020 = patent[patent['Year']=='2020']   # 4025건
# dup1 = patent_2020.duplicated(['Content'], keep='first')
# dup1.value_counts()   # 3864, 중복 161건
patent_2020 = patent_2020.drop_duplicates(['Content'])
patent_2020['Year'] = 2020   # int type
patent_2020   # 3864건

# 2021년 기준 (20년 + 21년 신규)
patent_2021 = patent[patent['Year']=='2021']   # 2246건
# dup2 = patent_2021.duplicated(['Content'], keep='first')
# dup2.value_counts()   # 2111, 중복 135건
patent_2021 = patent_2021.drop_duplicates(['Content'])
patent_2021   # 2111건
# 2021년 = 20년 + 21년 신규
patent_2021 = pd.concat([patent_2020, patent_2021], axis=0)
patent_2021['Year'] = 2021   # int type
patent_2021   # 5975건

,cmpID,Title,Content,Year
54,석경에이티,고순도 알루미나 분말 및 그 제조 방법,본 발명은 하기 단계를 포함하는 고순도 알루미나 제조방법을 제공한다 저순도 알루미...,2021
55,석경에이티,특정한 파장의 유해 블루라이트 을 차단하는 형광체 디스플레이용 투명 부재 또는...,본 발명은 특정 파장의 유해 블루 라이트 만 흡수하는 이하의 무기 투명 소재 상...,2021
56,석경에이티,대구경 분말의 제조 방법 및 이를 포함하는 화장료 조성물,본 발명은 에 를 투입하고 교반하는 단계 상기 교반 후 를 투입하고 교반하...,2021
93,지놈앤컴퍼니,항암 및 면역증강용 신규 타겟,본 발명은 암 치료 또는 예방용 약학적 조성물 면역증강용 약학적 조성물 항암제 스크...,2021
98,지놈앤컴퍼니,파우더 용기에 담지하여 사용 편리성 및 사용감이 개선된 기초 화장료 조성물 ...,본 발명은 파우더 용기에 적용되는 파우더 제형의 기초 화장료 조성물에 관한 것으로 ...,2021
...,...,...,...,...
88303,일지테크,스프링 와이어 프레임을 갖는 차량용 비성형 발포 패드,본 발명은 스프링 와이어 프레임을 갖는 차량용 비성형 발포 패드에 관한 것으로서 와...,2021
88304,일지테크,이종 소재 접합 장치 및 방법,본 발명은 금속 판재로 펄스 레이저를 조사하는 레이저 모듈 금속 판재를 가열하는 가...,2021
88307,일지테크,차량의 대시 패널 및 브레이크 내구성 측정장치,본 발명은 차량의 대시 패널 및 브레이크 내구성 측정장치에 관한 것으로서 브레이크 ...,2021
88328,일지테크,피어싱 용접장치 및 이를 이용한 피어싱 용접방법,본 발명은 피어싱 용접장치 및 이를 이용한 피어싱 용접방법에 관한 것으로 상부에 가...,2021


In [20]:
# 2020년, 2021년 한 테이블로 합치기
patent_concat = pd.concat([patent_2020, patent_2021], axis=0)

# 엑셀파일로 저장
patent_concat.to_excel('C:/Users/Chae/Desktop/SeSAC/Project/5. ESG_prj/2. 전처리/patent_preprocessing__final.xlsx', index=False)